In [1]:
from generator import LowShotGenerator
from data_utils import *
from sklearn.model_selection import train_test_split
import pandas as pd
from classifier import Classifier

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Initial Fetch Data - Perform once

In [2]:
data_obj = get_processed_data(num_files_to_fetch_data_from=1)
le = get_label_encoder(data_obj)

X, y = get_features_and_labels(data_obj)

fetching data from file #1


## Constants

In [3]:
n_clusters = 20
λ = .5
novel_disease_name = 'Hernia'
novel_disease_label = le.transform((novel_disease_name,))[0]
n = 2
n_samples_to_generate = 20

## Train a classifier on all diseases but one:

In [4]:
X_train, X_test, y_train, y_test = get_train_test_split_without_disease(X, y, novel_disease_name, data_obj)

cls = Classifier(n_classes=N_CLASSES - 1, n_epochs=1)
cls.fit(X_train, y_train)
print("accuracy acheived: %f" % cls.evaluate(X_test, y_test)[1])

Train on 3367 samples, validate on 375 samples
Epoch 1/1
416/416 [==============================] - 0s 37us/step
accuracy acheived: 0.112981


## Use the trained classifier and generate new examples:

In [5]:
X_train, X_test, y_train, y_test, n_samples_features, n_samples_integer_labels = get_train_test_split_with_n_samples_of_disease(X, y, novel_disease_name, data_obj, n)

generated_features = LowShotGenerator.get_generated_features(cls,
                                                             novel_disease_label,
                                                             n_samples_features,
                                                             n_clusters,
                                                             λ,
                                                             n_samples_to_generate)

Classifier is now non-trainable!
Loaded centroids from file
Generator summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 6144)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               3146240   
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
generator (Dense)            (None, 2048)              1050624   
Total params: 4,459,520
Trainable params: 4,459,520
Non-trainable params: 0
_________________________________________________________________
None

Whole model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)      

## Train a classifier on real n_samples of one disease

In [6]:
X_train, X_test, y_train, y_test, n_samples_features, n_samples_integer_labels = get_train_test_split_with_n_samples_of_disease(X, y, novel_disease_name, data_obj, n)

cls = Classifier(n_classes=N_CLASSES, n_epochs=1)
cls.fit(X_train, y_train)
print("accuracy acheived: %f" % cls.evaluate(X_test, y_test)[1])

Train on 2808 samples, validate on 312 samples
Epoch 1/1
1055/1055 [==============================] - 0s 36us/step
accuracy acheived: 0.088152


## Train a classifier on real n_samples + generated samples

In [7]:
generated_data_label = novel_disease_label
X_train, X_test, y_train, y_test = get_train_test_with_generated_data(X_train, X_test, y_train, y_test, generated_features, generated_data_label)

cls = Classifier(n_classes=N_CLASSES, n_epochs=1)
cls.fit(X_train, y_train)
print("accuracy acheived: %f" % cls.evaluate(X_test, y_test)[1])

18
(2048,)
(3120, 2048)
(3120, 15)
Train on 2824 samples, validate on 314 samples
Epoch 1/1
1055/1055 [==============================] - 0s 36us/step
accuracy acheived: 0.065403
